In [19]:
from google.colab import files
import pandas as pd
import io

upload_files = files.upload()

file_name = next(iter(upload_files))

df = pd.read_csv(io.StringIO(upload_files[file_name].decode('utf-8')), header=1, delimiter=";")

df

Saving default of credit card clients.csv to default of credit card clients (3).csv


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [20]:
df = df.drop('ID', axis = 1)
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [21]:
print(df["EDUCATION"].unique())

[2 1 3 5 4 6 0]


In [22]:
# Combine EDUCATION values 0, 5 and 6 into 4
# 1 = graduate school; 2 = university; 3 = high school; 4 = others.
df["EDUCATION"] = df["EDUCATION"].replace([0, 5, 6], 4)
df["EDUCATION"].unique()

array([2, 1, 3, 4])

In [23]:
print(df["MARRIAGE"].unique())

[1 2 3 0]


In [24]:
# Combining MARRIAGE column "0" values to 3
# 1 = married; 2 = single; 3 = others.
df["MARRIAGE"] = df["MARRIAGE"].replace(0, 3)
df["MARRIAGE"].unique()

array([1, 2, 3])

In [25]:
# Encoding EDUCATION column:
new_data = pd.get_dummies(df, columns=['EDUCATION', 'MARRIAGE'], drop_first=True)
new_data

,LIMIT_BAL,SEX,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_2,EDUCATION_3,EDUCATION_4,MARRIAGE_2,MARRIAGE_3
0,20000,2,24,2,2,-1,-1,-2,-2,3913,...,0,0,0,0,1,True,False,False,False,False
1,120000,2,26,-1,2,0,0,0,2,2682,...,1000,1000,0,2000,1,True,False,False,True,False
2,90000,2,34,0,0,0,0,0,0,29239,...,1000,1000,1000,5000,0,True,False,False,True,False
3,50000,2,37,0,0,0,0,0,0,46990,...,1200,1100,1069,1000,0,True,False,False,False,False
4,50000,1,57,-1,0,-1,0,0,0,8617,...,10000,9000,689,679,0,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,39,0,0,0,0,0,0,188948,...,5003,3047,5000,1000,0,False,True,False,False,False
29996,150000,1,43,-1,-1,-1,-1,0,0,1683,...,8998,129,0,0,0,False,True,False,True,False
29997,30000,1,37,4,3,2,-1,0,0,3565,...,22000,4200,2000,3100,1,True,False,False,True,False
29998,80000,1,41,1,-1,0,0,0,-1,-1645,...,1178,1926,52964,1804,1,False,True,False,False,False


In [26]:
print(df["SEX"].unique())

[2 1]


In [27]:
# Encoding SEX column:
df['SEX'] = df['SEX'] - 1
print(df["SEX"].unique())
# 0 = male, 1 = female

[1 0]


In [28]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']

pre = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical_features)],
    remainder='passthrough'
)

x = df.drop('default payment next month', axis = 1)
y = df['default payment next month']

In [29]:
from sklearn.preprocessing import StandardScaler

x_encoded = pre.fit_transform(x)
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_encoded)

In [30]:
from sklearn.model_selection import train_test_split

x = df.drop(columns=["default payment next month"])
y = df["default payment next month"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [31]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

y_train = y_train.reset_index(drop=True)
hist = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2, class_weight={ 0: 1.0, 1: 3.0 })

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5965 - loss: 932.6964 - val_accuracy: 0.5977 - val_loss: 203.2904
Epoch 2/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5861 - loss: 204.5204 - val_accuracy: 0.4906 - val_loss: 78.6735
Epoch 3/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6090 - loss: 80.0766 - val_accuracy: 0.4652 - val_loss: 25.5505
Epoch 4/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5993 - loss: 63.0446 - val_accuracy: 0.3913 - val_loss: 25.9696
Epoch 5/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5948 - loss: 29.3281 - val_accuracy: 0.7675 - val_loss: 9.2396
Epoch 6/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5957 - loss: 20.4503 - val_accuracy: 0.2756 - val_loss: 17.2538
Epoch 7/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5824 - loss: 12.7805 - val_accuracy: 0.6556 - val_loss: 4.9577
Epoch 8/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5932 - loss: 7.9786 - val_accuracy

In [32]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

y_pred = (model.predict(x_test) > 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7772 - loss: 0.7596
Test accuracy: 0.781166672706604
188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.78      1.00      0.88      4687
           1       0.50      0.00      0.00      1313

    accuracy                           0.78      6000
   macro avg       0.64      0.50      0.44      6000
weighted avg       0.72      0.78      0.69      6000

